In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.display import display
import sys
import os
import logging

sys.path.insert(0,'../src')
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 100

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import NaiveDE
import SpatialDE
from ipywidgets import IntProgress
IntProgress(10,max=100)

/home/users/jjzhu/.ve/space_met/lib/python3.6/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


IntProgress(value=10)

In [4]:
from io_utils import save_data_to_file, load_data_from_file
from general_utils import evaluate_rejections, norm_mtx

def read_trial_data(sim_dir, entry):
    lfn = os.path.join(sim_dir, entry['locs_fn'])
    locs = load_data_from_file(lfn, 'csv')
    dfn = os.path.join(sim_dir, entry['expr_fn'])
    data = load_data_from_file(dfn, 'csv')
    return locs, data
    
sim_dir = '/scratch/PI/sabatti/space_comp_sim/20191104'
sim_fn = os.path.join(sim_dir, 'sim_setup.csv')
sim_df = load_data_from_file(sim_fn, 'csv')
display(sim_df.head())

alpha = 0.05
meth_dir = 'result_spatialde'
cache=False

meth_dir = os.path.join(sim_dir, meth_dir)
os.makedirs(meth_dir, exist_ok=True)

eval_fn = 'eval_spatialde.csv'
eval_fn = os.path.join(sim_dir, eval_fn)
eval_df = pd.DataFrame()

for i, entry in sim_df.iterrows():
    locs, data = read_trial_data(sim_dir, entry)
    data = norm_mtx(data)
    sample_info = pd.DataFrame({'total_counts': data.sum(axis=1)})
#     dfm = NaiveDE.stabilize(data.T).T
#     res = NaiveDE.regress_out(sample_info, dfm.T, 'np.log(total_counts)').T
#     data = NaiveDE.regress_out(sample_info, norm_expr.T, 'total_counts').T
#     data = res
    
    assert locs.shape[0] == data.shape[0], 'Mismatch samples'
    fn = os.path.join(meth_dir, 'result_{}.csv'.format(i))
    if cache and os.path.exists(fn):
        result = load_data_from_file(fn, 'csv')
    else:
        result= SpatialDE.run(locs, data)
        save_data_to_file(result, fn, 'csv')   
        
    rej_idx = np.array(result.loc[result['qval'] < alpha].g).astype(int)
    nn_idx = np.arange(entry['n_per_reg'] * entry['n_regs'])
    evals = evaluate_rejections(set(rej_idx), set(nn_idx))
    print('{}-{}: Power: {:.4f}, FDP: {:.4f}'.format(entry['temp'], entry['seed'], 
                                              evals['Power'], evals['FDP']))
    eval_df = eval_df.append(pd.Series(evals, name=i))
save_data_to_file(eval_df, eval_fn, 'csv')

,seed,temp,n_per_reg,n_regs,locs_fn,expr_fn
0,0,linear,25,4,locs_2d.csv,sim_data/mtx_linear_0.csv
1,0,diaquad,25,4,locs_2d.csv,sim_data/mtx_diaquad_0.csv
2,0,radial,25,4,locs_2d.csv,sim_data/mtx_radial_0.csv
3,0,cosine,25,4,locs_2d.csv,sim_data/mtx_cosine_0.csv
4,1,linear,25,4,locs_2d.csv,sim_data/mtx_linear_1.csv


HBox(children=(IntProgress(value=0, description='Models: ', max=10, style=ProgressStyle(description_width='initial')), HTML(value='')))

HBox(children=(IntProgress(value=0, max=200), HTML(value='')))

HBox(children=(IntProgress(value=0, max=200), HTML(value='')))

HBox(children=(IntProgress(value=0, max=200), HTML(value='')))

...

HBox(children=(IntProgress(value=0, max=200), HTML(value='')))

HBox(children=(IntProgress(value=0, max=200), HTML(value='')))

HBox(children=(IntProgress(value=0, max=200), HTML(value='')))


linear-0: Power: 0.7500, FDP: 0.0000


AssertionError: Mismatch samples

In [40]:
result.tail()

,FSV,M,g,l,max_delta,max_ll,max_mu_hat,max_s2_t_hat,model,n,s2_FSV,s2_logdelta,time,BIC,max_ll_null,LLR,pval,qval
195,0.026747,4,38,0.766981,20.669312,1470.248034,-0.877454,0.002230,SE,1045,0.000347,0.391852,0.001966,-2912.688980,1466.081924,4.166110,0.041240,0.987656
196,0.013577,4,42,0.766981,41.269767,1434.977802,-0.976477,0.002383,SE,1045,0.000228,0.946915,0.001973,-2842.148515,1433.129868,1.847934,0.174024,0.987656
197,0.009934,4,53,0.766981,56.616267,1371.083931,0.770334,0.001417,SE,1045,0.000184,1.403470,0.001974,-2714.360774,1369.897809,1.186122,0.276113,0.987656
198,0.025059,4,60,0.766981,22.100310,1385.710404,0.851971,0.002111,SE,1045,0.000313,0.399760,0.001971,-2743.613720,1382.058922,3.651482,0.056019,0.987656
199,0.005066,4,71,0.766981,111.549286,1398.893056,0.847346,0.001420,SE,1045,0.000273,7.852686,0.001396,-2769.979022,1398.473254,0.419801,0.517036,0.987656


In [30]:
locs, data = read_trial_data(sim_dir, entry)
sample_info = pd.DataFrame({'total_counts': data.sum(axis=1)})
dfm = NaiveDE.stabilize(data.T).T
res = NaiveDE.regress_out(sample_info, dfm.T, 'np.log(total_counts)').T

/home/users/jjzhu/.ve/space_met/lib/python3.6/site-packages/NaiveDE/base.py:68: RuntimeWarning: invalid value encountered in log
  return np.log(expression_matrix + 1. / (2 * phi_hat[0]))


In [34]:
locs, data = read_trial_data(sim_dir, entry)
sample_info = pd.DataFrame({'total_counts': data.sum(axis=1)})
dfm = NaiveDE.stabilize(data.T).T
res = NaiveDE.regress_out(sample_info, dfm.T, 'np.log(total_counts)').T

/home/users/jjzhu/.ve/space_met/lib/python3.6/site-packages/NaiveDE/base.py:68: RuntimeWarning: invalid value encountered in log
  return np.log(expression_matrix + 1. / (2 * phi_hat[0]))


In [33]:
np.round(data)

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,1.0,4.0,2.0,4.0,3.0,3.0,3.0,1.0,3.0,3.0,...,3.0,3.0,2.0,4.0,3.0,3.0,5.0,4.0,4.0,2.0
1,0.0,2.0,2.0,4.0,2.0,3.0,1.0,0.0,2.0,3.0,...,3.0,5.0,2.0,3.0,5.0,3.0,1.0,3.0,4.0,4.0
2,0.0,1.0,2.0,3.0,3.0,1.0,1.0,2.0,0.0,3.0,...,3.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,2.0
3,2.0,1.0,2.0,4.0,1.0,0.0,2.0,2.0,0.0,1.0,...,3.0,1.0,4.0,4.0,4.0,2.0,3.0,1.0,1.0,4.0
4,2.0,2.0,3.0,4.0,3.0,1.0,2.0,3.0,3.0,2.0,...,1.0,2.0,4.0,2.0,3.0,3.0,3.0,3.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040,11.0,10.0,10.0,11.0,11.0,11.0,10.0,11.0,10.0,10.0,...,2.0,3.0,1.0,2.0,1.0,4.0,2.0,5.0,3.0,1.0
1041,9.0,12.0,11.0,12.0,11.0,11.0,13.0,11.0,10.0,11.0,...,3.0,5.0,3.0,5.0,4.0,4.0,4.0,3.0,3.0,2.0
1042,9.0,9.0,10.0,11.0,11.0,10.0,11.0,8.0,10.0,9.0,...,3.0,3.0,2.0,3.0,4.0,2.0,2.0,4.0,4.0,2.0
1043,4.0,5.0,7.0,7.0,6.0,5.0,3.0,6.0,5.0,6.0,...,3.0,3.0,2.0,3.0,3.0,3.0,2.0,3.0,3.0,2.0


In [16]:
result = results
rej_idx = np.array(result.loc[result['qval'] < 0.05].index).astype(int)
nn_idx = np.arange(entry['n_per_reg'] * entry['n_regs'])
evals = evaluate_rejections(set(rej_idx), set(nn_idx))
print('{}-{}: Power: {:.4f}, FDP: {:.4f}'.format(entry['temp'], entry['seed'], 
                                          evals['Power'], evals['FDP']))


linear-0: Power: 0.0100, FDP: 0.9901


In [18]:
result.head()

,FSV,M,g,l,max_delta,max_ll,max_mu_hat,max_s2_t_hat,model,n,s2_FSV,s2_logdelta,time,BIC,max_ll_null,LLR,pval,qval
0,2.061132e-09,4,100,0.014369,4.851652e+08,-1478.502121,3.039681,2.108866e-08,SE,1045,2.076978,3.539939e+17,0.001023,2984.811331,-1478.502361,0.000239,0.987656,0.987656
1,9.999546e-01,4,101,0.014369,4.539993e-05,-1480.555196,3.409182,1.249684e+01,SE,1045,13.955444,4.903673e+09,0.001015,2988.917480,-1480.921550,0.366354,0.544998,0.801468
2,9.999546e-01,4,102,0.014369,4.539993e-05,-1449.119188,2.985108,9.755394e+00,SE,1045,1109.070736,3.897060e+11,0.001021,2926.045465,-1449.236690,0.117502,0.731760,0.914157
3,9.999546e-01,4,104,0.014369,4.539993e-05,-1499.518693,3.183944,1.106415e+01,SE,1045,288.006808,1.012000e+11,0.001018,3026.844476,-1499.614856,0.096163,0.756484,0.922542
4,9.999546e-01,4,107,0.014369,4.539993e-05,-1527.450367,3.128348,1.077359e+01,SE,1045,1177.046397,4.135913e+11,0.001016,3082.707822,-1527.557979,0.107612,0.742879,0.914157


In [3]:
from io_utils import save_data_to_file, load_data_from_file
from general_utils import norm_mtx, evaluate_rejections

def read_trial_data(sim_dir, entry):
    lfn = os.path.join(sim_dir, entry['locs_fn'])
    locs = load_data_from_file(lfn, 'csv')
    dfn = os.path.join(sim_dir, entry['expr_fn'])
    data = load_data_from_file(dfn, 'csv')
    return locs, data
    
sim_dir = '/scratch/PI/sabatti/space_comp_sim/20191030'
sim_fn = os.path.join(sim_dir, 'sim_setup.csv')
sim_df = load_data_from_file(sim_fn, 'csv')
display(sim_df.head())

meth_dir = 'result_ours'
cache=True

meth_dir = os.path.join(sim_dir, meth_dir)
os.makedirs(meth_dir, exist_ok=True)

eval_fn = 'eval_ours.csv'
eval_fn = os.path.join(sim_dir, eval_fn)
eval_df = pd.DataFrame()


alpha= 0.05
knn = 8
n_perm = 10000

for i, entry in sim_df.iterrows():
    locs, data_norm = read_trial_data(sim_dir, entry)
    data_norm = norm_mtx(data_norm)
    assert locs.shape[0] == data_norm.shape[0], 'Mismatch samples'
    
    fn = os.path.join(meth_dir, 'result_{}.csv'.format(i))
    if cache and os.path.exists(fn):
        result = load_data_from_file(fn, 'csv')
        rej_idx = result.loc[result['reject']].index
    else:
       
        # save result to file
        save_data_to_file(result, fn, 'csv')        
    # evaluate for tracking
    nn_idx = np.arange(entry['n_per_reg'] * entry['n_regs'])
    evals = evaluate_rejections(set(rej_idx), set(nn_idx))
    print('{}-{}: Power: {:.4f}, FDP: {:.4f}'.format(entry['temp'], entry['seed'], 
                                              evals['Power'], evals['FDP']))
    
    eval_df = eval_df.append(pd.Series(evals, name=i))
save_data_to_file(eval_df, eval_fn, 'csv')

,seed,temp,n_per_reg,n_regs,locs_fn,expr_fn
0,0,linear,25,4,locs_2d.csv,sim_data/mtx_linear_0.csv
1,0,diaquad,25,4,locs_2d.csv,sim_data/mtx_diaquad_0.csv
2,0,radial,25,4,locs_2d.csv,sim_data/mtx_radial_0.csv
3,0,cosine,25,4,locs_2d.csv,sim_data/mtx_cosine_0.csv
4,1,linear,25,4,locs_2d.csv,sim_data/mtx_linear_1.csv


linear-0: Power: 1.0000, FDP: 0.0291
diaquad-0: Power: 0.9700, FDP: 0.0300
radial-0: Power: 1.0000, FDP: 0.0196
cosine-0: Power: 0.7600, FDP: 0.0617
linear-1: Power: 1.0000, FDP: 0.0099


AssertionError: Mismatch samples

In [7]:
locs.shape

(1045, 2)